In [1]:
%%bash
chia version
cdv --version

1.3.5
cdv, version 1.0.8


# Tic Tac Toe Coin

[tic-tac-toe puzzle](tic-tac-toe-ipynb) is an inner puzzle.

[p2_delegated_puzzle_or_hidden_puzzle.py](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py)

In [2]:
from chia.types.blockchain_format.program import Program
from cdv.util.load_clvm import load_clvm
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text

from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.')

In [3]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

from cdv.test import Network, Wallet

network: Network = await Network.create()

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

alice: 2000000000000


In [4]:
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(tic_tac_toe_puzzle, curried_tic_tac_toe_puzzle)

print(f'{board} {player}')
print(curried_tic_tac_toe_puzzle.get_tree_hash())
print(curried_tic_tac_toe_coin_puzzle.get_tree_hash())

# player 1 plays position 4
next_player = 2
position = 4
solution = Program.to([position, next_player])
curried_tic_tac_toe_puzzle = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(curried_tic_tac_toe_puzzle))
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(tic_tac_toe_puzzle, curried_tic_tac_toe_puzzle)
print(curried_tic_tac_toe_coin_puzzle.get_tree_hash())

# player 2 plays position 5
next_player = 1
position = 5
solution = Program.to([position, player])
curried_tic_tac_toe_puzzle = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(curried_tic_tac_toe_puzzle))
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(tic_tac_toe_puzzle, curried_tic_tac_toe_puzzle)
print(curried_tic_tac_toe_coin_puzzle.get_tree_hash())

# player 1 plays position 1
next_player = 2
position = 1
solution = Program.to([position, player])
curried_tic_tac_toe_puzzle = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(curried_tic_tac_toe_puzzle))
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(tic_tac_toe_puzzle, curried_tic_tac_toe_puzzle)
print(curried_tic_tac_toe_coin_puzzle.get_tree_hash())


[0, 0, 0, 0, 0, 0, 0, 0, 0] 1
15089968ec663f2f696effa21bc1b9d62045eb47ee4d5c9cb4a7fa7b2d901a7f
b2adeb99a3cec94f9dd403a6b1f2744700ac646fb1d89943545961a6465bc6a0
(a (q 2 (q 2 10 (c 2 (c (a 14 (c 2 (c 5 (c 11 (c () (c 23 ())))))) (c 11 ())))) (c (q ((a (i (any (a 12 (c 2 (c 9 (c 21 (c 45 (c 11 ())))))) (a 12 (c 2 (c 93 (c -67 (c 381 (c 11 ())))))) (a 12 (c 2 (c 765 (c 1533 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c 93 (c 765 (c 11 ())))))) (a 12 (c 2 (c 21 (c -67 (c 1533 (c 11 ())))))) (a 12 (c 2 (c 45 (c 381 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c -67 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 45 (c -67 (c 765 (c 11 ()))))))) (q 1 . 1) ()) 1) 2 (i (all (= 5 47) (= 11 47) (= 23 47)) (q 1 . 1) ()) 1) (c (a 8 (c 2 (c 5 (c 11 ())))) (c 5 ())) 2 (i (l 5) (q 4 (a (i (= 23 47) (q 2 (i (= () 9) (q . 11) (q 8 (q . "update error"))) 1) (q . 9)) 1) (a 14 (c 2 (c 13 (c 11 (c (+ 23 (q . 1)) (c 47 ()))))))) ()) 1) 1)) (c (q () () () () 1 () () () ()) (c (q . 2) 1)))
4c19c7c5ecccacd3dcb3cc08d5351a63a80b1e39